note: 
- We found that kendall implementation in the pandas differ from v1.3.1 and 1.1.5
- pandas does not have tau-c implementation
- we use kendall from the scipy package
- library version:
    - pandas: 1.3.1
    - scipy: 1.6.2

# flickr8k

In [35]:
import pandas as pd
import numpy as np

In [36]:
import scipy
from scipy import stats

def kendall_tau_b(x, y):
    tau, p_value = stats.kendalltau(x, y, variant="b")
    return tau

def kendall_tau_c(x, y):
    tau, p_value = stats.kendalltau(x, y, variant="c")
    return tau

scipy.__version__

'1.6.2'

In [37]:
# # official vilbertscore

# df = pd.read_csv("../results/flickr8k.csv")
# flickr_precision = df["precision"].to_list()
# flickr_recall = df["recall"].to_list()
# flickr_f1 = df["f1"].to_list()
    
# with open("../data/flickr8k/scores.pkl", "rb") as f:
#     human = pickle.load(f, encoding='latin1')
# human = [(x-1)/3 for x in human]

In [38]:
# custom vilbertscore
df = pd.read_csv("../data/processed/flickr8k/vilbertscore.csv")
flickr_precision = df["precision"].to_list()
flickr_recall = df["recall"].to_list()
flickr_f1 = df["f1"].to_list()
    
with open("../data/processed/flickr8k/annotations_avg.txt") as f:
    human = f.readlines()
    human = [ float(x.strip()) for x in human]
human = [(x-1)/3 for x in human]

df_final = pd.DataFrame([human, flickr_precision, flickr_recall, flickr_f1]).T
df_final = df_final.rename(columns={0:"human", 1:"precision", 2:"recall", 3:"f1"})
df_final.corr(method=kendall_tau_c)

,human,precision,recall,f1
human,1.000000,0.552884,0.526679,0.553806
precision,0.552884,1.000000,0.745920,0.881924
recall,0.526679,0.745920,1.000000,0.863986
f1,0.553806,0.881924,0.863986,1.000000


In [39]:
# df_final.corr(method=kendall_tau_b)

### flat annotation

In [24]:
# custom vilbertscore
df = pd.read_csv("../data/processed/flickr8k/vilbertscore.csv")
flickr_precision = df["precision"].to_list()
flickr_recall = df["recall"].to_list()
flickr_f1 = df["f1"].to_list()

flickr_precision_repeat3 = []
flickr_recall_repeat3 = []
flickr_f1_repeat_3 = []

for a, b, c in zip(flickr_precision, flickr_recall, flickr_f1):
    flickr_precision_repeat3.append(float(a))
    flickr_precision_repeat3.append(float(a))
    flickr_precision_repeat3.append(float(a))
    
    flickr_recall_repeat3.append(float(b))
    flickr_recall_repeat3.append(float(b))
    flickr_recall_repeat3.append(float(b))
    
    flickr_f1_repeat_3.append(float(c))
    flickr_f1_repeat_3.append(float(c))
    flickr_f1_repeat_3.append(float(c))
    
human_flat = []
with open("../data/processed/flickr8k/annotations.txt") as f:
    tmp = f.readlines()
    for item in tmp:
        a, b, c = item.strip().split(",")
        human_flat.append(float(a))
        human_flat.append(float(b))
        human_flat.append(float(c))
len(human_flat)

17466

In [25]:
df_final_flat = pd.DataFrame([human_flat, flickr_precision_repeat3, flickr_recall_repeat3, flickr_f1_repeat_3]).T
df_final_flat = df_final_flat.rename(columns={0:"human", 1:"precision", 2:"recall", 3:"f1"})
df_final_flat.corr(method=kendall_tau_c)

,human,precision,recall,f1
human,1.000000,0.540232,0.518112,0.541156
precision,0.540232,1.000000,0.745920,0.881924
recall,0.518112,0.745920,1.000000,0.863986
f1,0.541156,0.881924,0.863986,1.000000


In [26]:
# df_final_flat.corr(method=kendall_tau_b)

# capeval1k

In [27]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv("../data/raw/capeval1k/capeval1k_all_metrics.csv", index_col=False)
df = df.drop(["Unnamed: 0"], axis=1)

In [31]:
df_vs = pd.read_csv("../data/processed/capeval1k/vilbertscore.csv", index_col=False)
df_vs_f1 = df_vs.drop(["precision", "recall"], axis=1)
df_vs_f1 = df_vs_f1.rename(columns={"f1":"vilbertscore"})

df_clip = pd.read_csv("../data/processed/capeval1k/clipcore.csv", index_col=False)

In [32]:
df_merge = pd.concat([df, df_vs_f1, df_clip], axis=1)
df_merge.corr(method=kendall_tau_c)

,Human,b1,b4,rouge,meteor,cider,spice,umic,uniter,vilbertscore,clips,ref-clips
Human,1.000000,0.233369,0.237954,0.220303,0.288215,0.307178,0.279076,0.328425,0.298916,0.355099,0.270098,0.307777
b1,0.233369,1.000000,0.496028,0.754730,0.701726,0.628581,0.436585,0.225329,0.218973,0.442607,0.262552,0.310215
b4,0.237954,0.496028,1.000000,0.539021,0.550064,0.646615,0.394028,0.188705,0.174234,0.328371,0.210966,0.253249
rouge,0.220303,0.754730,0.539021,1.000000,0.670549,0.614650,0.434719,0.213494,0.210189,0.444972,0.241131,0.284344
meteor,0.288215,0.701726,0.550064,0.670549,1.000000,0.718165,0.524905,0.260215,0.261497,0.457551,0.312479,0.364420
cider,0.307178,0.628581,0.646615,0.614650,0.718165,1.000000,0.540019,0.239962,0.239124,0.455355,0.300344,0.358462
spice,0.279076,0.436585,0.394028,0.434719,0.524905,0.540019,1.000000,0.217322,0.220170,0.395942,0.285390,0.331005
umic,0.328425,0.225329,0.188705,0.213494,0.260215,0.239962,0.217322,1.000000,0.814596,0.277917,0.302507,0.316602
uniter,0.298916,0.218973,0.174234,0.210189,0.261497,0.239124,0.220170,0.814596,1.000000,0.261981,0.318233,0.324040
vilbertscore,0.355099,0.442607,0.328371,0.444972,0.457551,0.455355,0.395942,0.277917,0.261981,1.000000,0.332629,0.391940


In [ ]:
# import json

# with open("../data/processed/capeval1k/cand.txt") as f:
#     cap_test = f.readlines()
#     cap_test = [x.strip() for x in cap_test]

# with open("../data/processed/capeval1k/capeval_clip_result.json") as f:
#     clip = json.load(f)
# clip[0]

# list_clips = []
# list_refclips = []
# for check, y in zip(cap_test, clip):
#     if check == y["candiate_caption"]:
#         list_clips.append(y["CLIP-S"])
#         list_refclips.append(y["RefCLIP-S"])
        
# df = pd.DataFrame([list_clips, list_refclips]).T
# df = df.rename(columns={0:"clips", 1:"ref-clips"})
# df.to_csv("../data/processed/capeval1k/clipcore.csv", index=False)